In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib as plt
from pathlib import Path

In [ ]:
file_path = r'C:\Users\justi\Documents\Projects\Data\Baseball_databank'

teams_path = file_path + '\Teams.csv'
salary_path = file_path + '\Salaries.csv'
batting_path = file_path + '\Batting.csv'

In [ ]:
teams_df = pd.read_csv(teams_path)
teams_df = teams_df[teams_df['yearID'] >= 2000]
teams_df.info()

In [ ]:
salary_df = pd.read_csv(salary_path)
salary_df = salary_df[salary_df['yearID'] >= 1985]
salary_df.info()

In [ ]:
# How have salaries evolved by division over the years?
# Sum of salaries by division by year, then dig in by team

# .reset_index() converts back to a pandas dataframe, otherwise it would have been a series

team_salary_by_year_df = salary_df.groupby(['yearID','teamID','lgID'])['salary'].sum().reset_index()
team_salary_by_year_df.head()

In [ ]:
# cool reference material for graphs here:
# https://medium.com/dataexplorations/reproducing-the-baseball-salary-visualization-from-fivethirtyeight-in-matplotlib-1af449beb409
# Lets try and recreate!

salary_data_df['salary_zscore'] = team_salary_by_year_df.groupby(['yearID','lgID'])['salary'].transform(
    lambda x: (x - x.mean()) / x.std()
)

salary_data_df.head()